In [2]:
!pip install transformer_lens

import transformer_lens
from transformer_lens import utils
import torch
import tqdm
from functools import partial

In [4]:
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-xl")

logits, activations = model.run_with_cache("Hello World my dear")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-xl into HookedTransformer


In [6]:
model.generate("I am going to kill you for this outrageous", max_new_tokens=100)

  0%|          | 0/100 [00:00<?, ?it/s]

"I am going to kill you for this outrageous comment by a Chicago conservative who unaccountably praised United Nations Secretary-General Ban Ki-Moon, over the question of reproductive procedures, on CNN's Reliable Sources.\n\nBan is a fantastic leader, I love him. He was appointed by the current U.S. president to serve his entire tenure of office in 2017/18, and it has over 14 months left. However, there are many around him who will say otherwise.\n\nI'd bet my nomination as president that if Ban"

In [153]:
positive_prompt = "Love "
negative_prompt = "Hate"

print(model.to_tokens(positive_prompt))
print(model.to_tokens(negative_prompt))

tensor([[50256, 18565,   220]], device='cuda:0')
tensor([[50256,    39,   378]], device='cuda:0')


In [154]:

_, activations_love = model.run_with_cache(positive_prompt)
_, activations_hate = model.run_with_cache(negative_prompt)

In [155]:

layer_key = 'blocks.6.hook_resid_pre'
print(activations_love[layer_key].shape)
steering_vector = activations_love[layer_key][0] - activations_hate[layer_key][0]
print(steering_vector.shape)

torch.Size([1, 3, 1600])
torch.Size([3, 1600])


In [165]:


prompt = "I hate you because"
tokens= model.to_tokens(prompt)
act_vector = steering_vector

def head_ablation_hook(
    value,
    hook,
    act_vector
):
    if value.shape[1] == 1:
        return  # caching in model.generate for new tokens

    a = act_vector.shape[0]

    print(f"Shape of the value tensor: {value.shape} and shape of act: {act_vector.shape}")
    value[:, :a, :] +=  act_vector*4
    print(f"after Shape of the value tensor: {value.shape}")
    return value

model.reset_hooks()


In [167]:
fwd_hooks=[(
        layer_key, 
        partial(head_ablation_hook, act_vector=act_vector)
        )]
empty_hooks = []
with model.hooks(fwd_hooks=fwd_hooks):
    print(model.generate(prompt, max_new_tokens=50, do_sample=True))
model.reset_hooks()

  0%|          | 0/50 [00:00<?, ?it/s]

Shape of the value tensor: torch.Size([1, 5, 1600]) and shape of act: torch.Size([3, 1600])
after Shape of the value tensor: torch.Size([1, 5, 1600])
I hate you because you've don't bother to let your cousin know that the have high schooler listen, same as Yuigahama during the Tsushima trip, he messed it up because he hasn't had any Vlogs before. The other day, he


  0%|          | 0/100 [00:00<?, ?it/s]

"I am going to kill you for this outrageous award, right? Comparing Liverpool's youngsters to the team that has recently won the Premier League does seem completely done and batty and was the most common reason given by sports journalists & pundits in the hours & days leading up to this game a month ago. Yes of course Kolo Toure survived the incident and played, but in hindsight, the former Ivory Coast international's antics stole the day for Roberto Firmino. But fair enough.\n\nLiverpool attribute their semi-final victory in the"